# Create N-Gram List from Excel Files

This notebook allows the user to enter a directory containing pre-processed Excel files after passing through an LLM in order to grab the distinct list of n-grams created when doing n-gram tracing. This will be used to filter out past results but also for future iterations this idea will be used prior to sending requests to an LLM.

In the future i will process all of the documents in n-gram tracing and output a file similar to this containing all possible n-grams in common. Then i will create a filtered list to remove some before processing them next time around and keep iterating on this list with each corpus.

Done to save time and money if using OpenAI as i feel the main 2-grams will be featured reguoarly.

In [6]:
import ast
import threading
import os
import time

import pandas as pd

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
doc_dir = '/Volumes/BCross/paraphrase examples slurm/Wiki-test-auto/'

output_path = "/Volumes/BCross/paraphrase examples slurm/wiki-phrase-list-raw.xlsx"

In [3]:
def get_sorted_distinct_references(doc_loc, sheet_name='no context'):
    """
    Reads a single Excel sheet, extracts distinct reference phrases, 
    parses the tokens list, and sorts the output by token count and phrase length.
    """

    # Only read the required columns for speed and memory efficiency
    use_cols = ['phrase_type', 'phrase', 'tokens']

    try:
        no_context = pd.read_excel(doc_loc, sheet_name=sheet_name, usecols=use_cols)
    except Exception as e:
        # Return empty DataFrame if the file or sheet can't be read
        print(f"⚠️ Failed to read {doc_loc}: {e}")
        return pd.DataFrame(columns=['phrase', 'tokens', 'num_tokens'])

    # Filter only the rows where phrase_type == 'reference'
    references = no_context[no_context['phrase_type'] == 'reference']

    # Remove duplicates and reset index
    references = references[['phrase', 'tokens']].drop_duplicates().reset_index(drop=True)

    # Safely parse stringified lists (e.g., "['a','b']") into Python lists
    def parse_tokens(x):
        try:
            return ast.literal_eval(x) if isinstance(x, str) else x
        except Exception:
            return []

    references['tokens'] = references['tokens'].map(parse_tokens)

    # Count number of tokens in each list
    references['num_tokens'] = references['tokens'].map(len)

    # Also compute phrase length in characters (for secondary sorting)
    references['phrase_len'] = references['phrase'].str.len()

    # Sort by number of tokens first, then phrase length
    sorted_refs = references.sort_values(by=['num_tokens', 'phrase_len']).reset_index(drop=True)

    # Remove temporary sorting column before returning
    return sorted_refs.drop(columns='phrase_len')


In [4]:
def get_sorted_ngrams_dir(doc_dir, sheet_name='no context'):
    """
    Loops through all Excel files in a directory, extracts distinct reference phrases 
    from each file in parallel, merges them, and returns a sorted, deduplicated DataFrame.
    Progress is shown as percentages (10%, 20%, ..., 100%).
    """

    excel_dir = Path(doc_dir)

    # Collect all Excel files (skip temporary lock files)
    excel_files = sorted([f for f in excel_dir.glob("*.xlsx") if not f.name.startswith("~$")])
    total_files = len(excel_files)
    print(f"🟢 Processing {total_files} Excel files in parallel...")

    # Shared state for progress tracking
    progress_state = {'count': 0, 'next_threshold': 10}
    lock = threading.Lock()

    def load_and_track_progress(excel_path):
        """Loads and processes a single Excel file, updating progress."""
        try:
            return get_sorted_distinct_references(excel_path, sheet_name=sheet_name)
        finally:
            with lock:
                progress_state['count'] += 1
                pct = (progress_state['count'] / total_files) * 100

                # Print at clean 10% intervals
                if pct >= progress_state['next_threshold']:
                    print(f"Progress: {int(progress_state['next_threshold'])}%")
                    progress_state['next_threshold'] += 10

    # Run file processing in parallel
    phrases_dataframe_list = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(load_and_track_progress, f) for f in excel_files]
        for future in as_completed(futures):
            result = future.result()
            if not result.empty:
                phrases_dataframe_list.append(result)

    print("✅ All files processed.")

    # Combine all results into one DataFrame
    combined = pd.concat(phrases_dataframe_list, ignore_index=True)

    # Convert lists to tuples so they can be hashed (for drop_duplicates)
    combined['tokens'] = combined['tokens'].map(lambda x: tuple(x) if isinstance(x, list) else x)

    # Drop duplicates across all files
    combined = combined[['phrase', 'tokens', 'num_tokens']].drop_duplicates().reset_index(drop=True)

    # Compute phrase length for sorting
    combined['phrase_len'] = combined['phrase'].str.len()

    # Sort by number of tokens first, then phrase length
    sorted_combined = combined.sort_values(by=['num_tokens', 'phrase_len']).reset_index(drop=True)

    # Drop helper column before returning
    return sorted_combined.drop(columns='phrase_len')

In [ ]:
base_data_dir = "/Volumes/BCross/av_datasets_experiments/ngram_masking"

data_types = ["test", "training"]
corpuses = ["ACL", "Enron", "Perverted Justice", "StackExchange",
            "The Telegraph", "TripAdvisor", "Wiki"]
paraphrasing_models = ["ModernBERT-base", "ModernBERT-large"]
scoring_models = ["gpt2"]

full_phrase_list = []

for dt in data_types:
    print(f"Working on {dt} data")
    for cp in corpuses:
        print(f"Working on the {cp} corpus")
        for pm in paraphrasing_models:
            print(f"Working on the {pm} paraphrasing model")
            for sm in scoring_models:
                print(f"Working on the {sm} scoring model")
                
                data_dir = f"{base_data_dir}/{dt}/{cp}/{pm}/{sm} results/raw"
                
                if not os.path.isdir(data_dir):
                    print(f"Skipping (missing): {data_dir}")
                    continue
                
                reference_phrases = get_sorted_ngrams_dir(data_dir)
                
                full_phrase_list.append(reference_phrases)
                
reference_ngrams = pd.concat(full_phrase_list, ignore_index=True)              

In [10]:
base_data_dir = "/Volumes/BCross/av_datasets_experiments/ngram_masking"

data_types = ["test", "training"]
corpuses = ["ACL", "Enron", "Perverted Justice", "StackExchange",
            "The Telegraph", "TripAdvisor", "Wiki"]
paraphrasing_models = ["ModernBERT-base", "ModernBERT-large"]
scoring_models = ["gpt2"]

full_phrase_list = []
skipped = 0
processed = 0

t0_total = time.perf_counter()

for dt in data_types:
    t0_dt = time.perf_counter()
    print(f"\nWorking on {dt} data")

    for cp in corpuses:
        t0_cp = time.perf_counter()
        print(f"  Working on the {cp} corpus")

        for pm in paraphrasing_models:
            t0_pm = time.perf_counter()
            print(f"    Working on the {pm} paraphrasing model")

            for sm in scoring_models:
                t0_combo = time.perf_counter()
                print(f"      Working on the {sm} scoring model")

                # NOTE: keeps your intended "gpt2 results/raw" directory naming
                data_dir = f"{base_data_dir}/{dt}/{cp}/{pm}/{sm} results/filtered_inc_rank"

                if not os.path.isdir(data_dir):
                    skipped += 1
                    print(f"      Skipping (missing): {data_dir}")
                    continue

                t0_read = time.perf_counter()
                reference_phrases = get_sorted_ngrams_dir(data_dir)
                t1_read = time.perf_counter()

                full_phrase_list.append(reference_phrases)
                processed += 1

                t1_combo = time.perf_counter()
                print(
                    f"      Done: read={t1_read - t0_read:.3f}s, "
                    f"combo_total={t1_combo - t0_combo:.3f}s"
                )

            t1_pm = time.perf_counter()
            print(f"    {pm} subtotal: {t1_pm - t0_pm:.3f}s")

        t1_cp = time.perf_counter()
        print(f"  {cp} subtotal: {t1_cp - t0_cp:.3f}s")

    t1_dt = time.perf_counter()
    print(f"{dt} subtotal: {t1_dt - t0_dt:.3f}s")

t0_concat = time.perf_counter()
reference_ngrams = (
    pd.concat(full_phrase_list, ignore_index=True)
    if full_phrase_list
    else pd.DataFrame()
)
t1_concat = time.perf_counter()

t1_total = time.perf_counter()
print("\n=== Summary ===")
print(f"Processed: {processed}")
print(f"Skipped:   {skipped}")
print(f"Concat:    {t1_concat - t0_concat:.3f}s")
print(f"Total:     {t1_total - t0_total:.3f}s")



Working on test data
  Working on the ACL corpus
    Working on the ModernBERT-base paraphrasing model
      Working on the gpt2 scoring model
      Skipping (missing): /Volumes/BCross/av_datasets_experiments/ngram_masking/test/ACL/ModernBERT-base/gpt2 results/filtered_inc_rank
    ModernBERT-base subtotal: 0.036s
    Working on the ModernBERT-large paraphrasing model
      Working on the gpt2 scoring model
      Skipping (missing): /Volumes/BCross/av_datasets_experiments/ngram_masking/test/ACL/ModernBERT-large/gpt2 results/filtered_inc_rank
    ModernBERT-large subtotal: 0.004s
  ACL subtotal: 0.041s
  Working on the Enron corpus
    Working on the ModernBERT-base paraphrasing model
      Working on the gpt2 scoring model
🟢 Processing 200 Excel files in parallel...
Progress: 10%
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%
✅ All files processed.
      Done: read=15.939s, combo_total=15.957s
    ModernBE

In [11]:
reference_ngrams

,phrase,tokens,num_tokens
0,as a,"(Ġas, Ġa)",2
1,on a,"(Ġon, Ġa)",2
2,to a,"(Ġto, Ġa)",2
3,be a,"(Ġbe, Ġa)",2
4,in a,"(Ġin, Ġa)",2
...,...,...,...
504,in the future,"(Ġin, Ġthe, Ġfuture)",3
505,because of the,"(Ġbecause, Ġof, Ġthe)",3
506,", i don't","(,, Ġi, Ġdon, 't)",4
507,i'm not sure,"(Ġi, 'm, Ġnot, Ġsure)",4


In [7]:
reference_phrases = get_sorted_ngrams_dir(doc_dir)

🟢 Processing 661 Excel files in parallel...
Progress: 10%
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%
✅ All files processed.


In [8]:
reference_phrases

,phrase,tokens,num_tokens
0,7 is,"(7, Ġis)",2
1,s of,"(s, Ġof)",2
2,'s a,"('s, Ġa)",2
3,i am,"(i, Ġam)",2
4,'t a,"('t, Ġa)",2
...,...,...,...
3021,in the interest of not starting a flame war or...,"(in, Ġthe, Ġinterest, Ġof, Ġnot, Ġstarting, Ġa...",72
3022,my advice to mr asquith is to stay away from w...,"(my, Ġadvice, Ġto, Ġmr, Ġas, qu, ith, Ġis, Ġto...",88
3023,".\ni was unaware of wikipedia, as i do not ref...","(.Ċ, i, Ġwas, Ġunaware, Ġof, Ġwikipedia, ,, Ġa...",154
3024,"indeed, this is pure logic just think about th...","(inde, ed, ,, Ġthis, Ġis, Ġpure, Ġlogic, Ġjust...",300


In [9]:
# reference_phrases.to_excel(output_path, index=False)

In [18]:
def find_phrase_or_tokens_in_dir(doc_dir, sheet_name='no context', target=None):
    """
    Searches through all Excel files in a directory (in parallel) 
    to find which ones contain a specific phrase or a specific list/tuple of tokens.
    Progress is shown as percentages (10%, 20%, ..., 100%).
    """

    if target is None:
        raise ValueError("You must provide a target phrase or tuple of tokens.")

    # Determine if this is a token or phrase search
    is_token_search = isinstance(target, (list, tuple))
    target_list = list(target) if is_token_search else str(target)

    excel_dir = Path(doc_dir)
    excel_files = sorted([f for f in excel_dir.glob("*.xlsx") if not f.name.startswith("~$")])
    total_files = len(excel_files)

    print(f"🔍 Searching {total_files} files for {'tokens' if is_token_search else 'phrase'}: {target_list}")

    # Shared progress tracker
    progress_state = {'count': 0, 'next_threshold': 10}
    lock = threading.Lock()

    def check_file(excel_path):
        """Worker function for one file."""
        try:
            df = get_sorted_distinct_references(excel_path, sheet_name=sheet_name)
            if df.empty:
                return None

            # Token-based or phrase-based matching
            if is_token_search:
                token_tuples = set(df['tokens'].map(tuple))
                if tuple(target_list) in token_tuples:
                    return excel_path.name
            else:
                phrases_set = set(df['phrase'])
                if target_list in phrases_set:
                    return excel_path.name

        except Exception as e:
            print(f"⚠️ Failed to process {excel_path.name}: {e}")
        finally:
            with lock:
                progress_state['count'] += 1
                pct = (progress_state['count'] / total_files) * 100

                # Print at clean 10% increments
                if pct >= progress_state['next_threshold']:
                    print(f"Progress: {int(progress_state['next_threshold'])}%")
                    progress_state['next_threshold'] += 10

        return None

    matched_files = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_file, f) for f in excel_files]
        for future in as_completed(futures):
            result = future.result()
            if result:
                matched_files.append(result)

    print("✅ Search complete.")
    return matched_files

In [41]:
common_phrase_doc_list = find_phrase_or_tokens_in_dir(
    doc_dir,
    sheet_name='no context',
    target=(',', 'Ġi', "'d")

)
common_phrase_doc_list

🔍 Searching 661 files for tokens: [',', 'Ġi', "'d"]
Progress: 10%
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%
✅ Search complete.


['jasper_deng_text_2 vs jasper_deng_text_4.xlsx',
 'pinkampersand_text_2 vs pinkampersand_text_4.xlsx',
 'pinkampersand_text_3 vs pinkampersand_text_4.xlsx',
 'pinkampersand_text_3 vs pro_lick_text_1.xlsx',
 'pro_lick_text_3 vs pro_lick_text_1.xlsx',
 'stephenbuxton_text_1 vs stillstanding_247_text_5.xlsx']